In [3]:
from datasets import load_dataset

/home/juyeon/miniconda3/envs/openai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# datasets
dataset = load_dataset("detection-datasets/fashionpedia")

Generating val split: 100%|██████████| 1158/1158 [00:00<00:00, 6073.71 examples/s]


In [9]:
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
import chromadb

In [10]:
chroma_client = chromadb.PersistentClient("./fvector")

In [11]:
chroma_client = chromadb.PersistentClient("./fvector")
image_loader = ImageLoader()

In [ ]:
#pip install open-clip-torch

CLIP = OpenCLIPEmbeddingFunction()

In [15]:
image_vdb = chroma_client.get_or_create_collection(
        name="image", embedding_function=CLIP, data_loader=image_loader)

In [20]:
import os

ids = []
uris =[]
roots = "./data/fashion_dataset/fashion_dataset/"

for idx, filename in enumerate(sorted(os.listdir(roots))):
    if filename.endswith(".png"):
        uris.append(roots+filename)
        ids.append(str(idx))

image_vdb.add(ids=ids, uris=uris)

### Translator

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [23]:
def translate(text):
    model  = ChatOpenAI(model='gpt-4.1', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', "You are a translator. Translate the following text to English"),
            ('user', "{text}")
        ]
    )


    chain = prompt | model | StrOutputParser()
    return chain.invoke({'text' : text})

In [26]:
query = translate("검은색 티셔츠와 청바지 추천")

In [27]:
tmp = image_vdb.query(
    query_texts=[query],
    n_results=2,
    include=['uris', 'distances']
)

In [28]:
tmp

{'ids': [['782', '510']],
 'embeddings': None,
 'documents': None,
 'uris': [['./data/fashion_dataset/fashion_dataset/image_802.png',
   './data/fashion_dataset/fashion_dataset/image_558.png']],
 'included': ['uris', 'distances'],
 'data': None,
 'metadatas': None,
 'distances': [[1.4392257928848267, 1.4442415237426758]]}

In [29]:
def query_db(image_vdb, query, results=2):
    # 주어진 쿼리를 실행하고, 상위 결과 반환
    return image_vdb.query(
        query_texts=[query],
        n_results=results,
        include=['uris', 'distances'])


In [31]:
def setup_chain():
    gpt4 = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    parser = StrOutputParser()
    image_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful fashion and styling assistant. Answer the user's question using the given image context with direct references to parts of the images provided. Maintain a more conversational tone, don't make too many lists. Use markdown formatting for highlights, emphasis, and structure."),
            ("user", [
                {"type": "text", "text": "What are some ideas for styling {user_query}"},
                {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
                {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_2}"},
            ]),
        ])


    return image_prompt | gpt4 | parser

### img를 넘겨줄 때 base64 encoding

In [32]:
import base64

In [45]:
def setup_chain():
    gpt4 = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
    parser = StrOutputParser()
    image_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful fashion and styling assistant. Answer the user's question using the given image context with direct references to parts of the images provided. Maintain a more conversational tone, don't make too many lists. Use markdown formatting for highlights, emphasis, and structure."),
            ("user", [
                {"type": "text", "text": "What are some ideas for styling {user_query}"},
                {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
                {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_2}"},
            ]),
        ])


    return image_prompt | gpt4 | parser

In [46]:
vision_chain = setup_chain()

In [47]:
def format_prompt_inputs(data, user_query):
    inputs = {}


    inputs['user_query'] = user_query


    image_path_1 = data['uris'][0][0]
    image_path_2 = data['uris'][0][1]


    with open(image_path_1, 'rb') as image_file:
        image_data_1 = image_file.read()
    inputs['image_data_1'] = base64.b64encode(image_data_1).decode('utf-8')


    with open(image_path_2, 'rb') as image_file:
        image_data_2 = image_file.read()
    inputs['image_data_2'] = base64.b64encode(image_data_2).decode('utf-8')


    return inputs

In [48]:
ko_query = "바지하고 티셔츠를 파란색으로 추천"
en_query = translate(ko_query)

In [49]:
results = query_db(image_vdb, en_query)
prompt_input = format_prompt_inputs(results, en_query)
response_en = vision_chain.invoke(prompt_input)

In [52]:
response_en

'Styling blue pants and a t-shirt can be both fun and versatile! Here are some ideas inspired by the images you provided:\n\n### Casual Chic\n- **Layering**: Consider adding a lightweight jacket or a denim shirt over your t-shirt. This adds depth and can be easily removed if it gets warm.\n- **Footwear**: Sneakers or casual loafers work well. In the first image, the model pairs blue pants with sporty shoes, which gives a relaxed vibe.\n\n### Smart Casual\n- **Button-Up Option**: Like in the second image, you could opt for a short-sleeve button-up shirt in a complementary shade of blue or a contrasting color. This elevates the look while keeping it casual.\n- **Accessories**: A stylish watch or a simple bracelet can add a touch of sophistication without being over the top.\n\n### Color Play\n- **Contrasting Colors**: Pair your blue pants with a t-shirt in a contrasting color, like white or a light gray. This creates a fresh and clean look.\n- **Patterned T-Shirts**: If you want to add s

In [55]:
def translate(text, target):
    model  = ChatOpenAI(model='gpt-4.1', temperature=0)
    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', f"You are a translator. Translate the following text to {target}"),
            ('user', "{text}")
        ]
    )


    chain = prompt | model | StrOutputParser()
    return chain.invoke({'text' : text})

In [56]:
rt  = translate(response_en, 'korean')

In [57]:
rt

'파란 바지와 티셔츠 스타일링은 재미있고 다양하게 연출할 수 있습니다! 제공해주신 이미지를 참고해 몇 가지 아이디어를 소개합니다.\n\n### 캐주얼 시크\n- **레이어링**: 티셔츠 위에 가벼운 재킷이나 데님 셔츠를 걸쳐보세요. 스타일에 깊이를 더해주고, 더워지면 쉽게 벗을 수 있습니다.\n- **신발**: 스니커즈나 캐주얼 로퍼가 잘 어울립니다. 첫 번째 이미지에서 모델은 파란 바지에 스포티한 신발을 매치해 편안한 분위기를 연출했습니다.\n\n### 스마트 캐주얼\n- **버튼업 셔츠**: 두 번째 이미지처럼, 파란색 계열이나 대비되는 색상의 반팔 버튼업 셔츠를 선택해보세요. 캐주얼하면서도 한층 세련된 느낌을 줍니다.\n- **액세서리**: 세련된 시계나 심플한 팔찌로 과하지 않게 멋을 더할 수 있습니다.\n\n### 컬러 플레이\n- **대비 색상**: 파란 바지에는 흰색이나 연회색 티셔츠처럼 대비되는 색상을 매치해보세요. 산뜻하고 깔끔한 느낌을 줍니다.\n- **패턴 티셔츠**: 좀 더 개성을 더하고 싶다면, 파란색이나 바지와 어울리는 색상이 들어간 패턴 티셔츠를 선택해보세요.\n\n### 계절별 연출\n- **쌀쌀한 날씨엔 레이어링**: 날씨가 선선할 때는 진한 파란색의 가벼운 스웨터나 후디를 매치해보세요. 첫 번째 이미지에서처럼 컬러 블로킹으로 다양한 파란색을 조합하는 것도 좋은 방법입니다.\n\n### 마무리 포인트\n- **그루밍**: 깔끔하게 다듬은 수염이나 헤어스타일은 전체적인 인상을 한층 업그레이드해줍니다. 두 이미지 모두에서 확인할 수 있습니다.\n- **자신감**: 무엇을 입든 자신감 있게! 이것이 최고의 액세서리입니다.\n\n이 아이디어들을 자유롭게 믹스매치해서 자신만의 스타일을 완성해보세요!'

---

In [59]:
from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent
from langchain import hub

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

from langchain.tools.retriever import create_retriever_tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_openai import ChatOpenAI
import os

In [60]:
openai = ChatOpenAI(model='gpt-4o-mini', temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")

/home/juyeon/miniconda3/envs/openai/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [63]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [64]:
# naver news
loader = WebBaseLoader(
    "https://news.naver.com/"
)

docs = loader.load()

In [65]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [70]:
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()

In [71]:
retriever_tool = create_retriever_tool(
    retriever, "naver_news_search", "네이버 뉴스정보가 저장된 벡터 DB, 당일 뉴스 검색 가능"
)

In [72]:
arxiv_wrapper = ArxivAPIWrapper(
    top_k_results=1, doc_content_chars_max=200, load_all_available_meta=False,)

arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

tools = [wiki, retriever_tool, arxiv]

In [73]:
agent = create_openai_tools_agent(llm=openai, tools=tools, prompt=prompt)

In [74]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [75]:
result = agent_executor.invoke({'input' : '오늘 증시 관련 주요 소식을 알려줘'})



> Entering new AgentExecutor chain...

Invoking: `naver_news_search` with `{'query': '증시'}`


구독





[제약사 리포트] 종근당, 2분기 매출 新고점 찍고도 고심…수익성 반등 요소는



[고려아연 TMC 투자] '원료팀·R&D' 물밑 공조 있었다


[증권가 분리과세 임팩트]② 미래에셋증권, IMA 지정 효과는 '미지수'


[보험사 보안.zip]③ 한화생명, 정보유출 '0건'…IT예산 12% 투입 효과











연합뉴스
08월 06일 16:11

구독





김건희 특검 조사 호칭은 '피의자'…"예상보다 길어질 가능성"(종합)



정청래號 검찰개혁특위 본격 가동…"골든타임 놓치면 개혁 좌초"


포스코이앤씨 면허취소 땐 28년 만에 첫 사례…건설업계 '초긴장'


충격파 낮췄지만 한국도 내일부터 트럼프 상호관세 영향권











YTN
08월 06일 16:20

구독




영상

김건희 조사 3시간 넘겨..."아무것도 아닌 저, 죄송"



이 대통령 "이춘석 의혹, 엄정 수사하라"...휴가 중 '긴급 지시'


'차명 거래 의혹' 이춘석 후폭풍..."제명" "국기 문란"


文 전 대통령, 대통령실에 "조국 사면·복권해야" 건의











MBN
08월 06일 16:25

구독





특검 "김건희, 진술 거부 안 해…전체 조사 분량 절반 넘겨"



문재인 전 대통령 "조국, 사면·복권해야"…대통령실에 의사 전달


1,000만 탈모인 귀 쫑긋…'탈모 예방' 식물 찾았다


홍준표 "윤석열, 국군통수권자로서 '격노'할 수 있어…뒤처리는 졸렬"











문화일보
08월 06일 15:59

구독





옥중 규탄했던 윤석열, 아내 김건희 출석 소식에 ‘침울’



초등생딸 폰 열어보니 “보고 싶어” 태권도 관장 보낸 수백통 메시지


‘억대 빚’ 아내와 이혼, 알고보니 16년 키운 자식도 ‘남남’


[속보]“롯데호텔갔나요 조선

In [76]:
print(result)

{'input': '오늘 증시 관련 주요 소식을 알려줘', 'output': '오늘 증시 관련 주요 소식은 다음과 같습니다:\n\n1. **종근당**: 2분기 매출이 신기록을 세웠지만, 수익성 반등에 대한 고민이 깊어지고 있습니다.\n\n2. **미래에셋증권**: 분리과세의 임팩트에 대해 논의되고 있으며, IMA 지정 효과는 아직 미지수로 평가되고 있습니다.\n\n3. **한화생명**: 정보 유출이 없었다고 발표하며, IT 예산의 12%를 투입한 효과를 강조하고 있습니다.\n\n이 외에도 증시와 관련된 다양한 이슈들이 보도되고 있으니, 추가적인 정보가 필요하시면 말씀해 주세요!'}
